In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.optimize as opt
from mpl_toolkits.mplot3d import Axes3D
from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D
from operator import itemgetter
from random_traj import randaniso, randconv, multrandaniso, multrandconv, randsamp, multrandsamp, randall, multrandall
from random_traj import randanisorot, multrandanisorot
from ellipsoid import maxtraj, mvee, plot_mvee, plot_mveeoverl, rotmat, rotmat2, plot_mveeoverl2
from ellipsoid import ellipsax, MSDS, andiff, plot_anisoMSDS, plot_anisodiff
import random
import numpy as np
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D
from ellipsoid import extrema, enclosed_MSD

pi = np.pi
sin = np.sin
cos = np.cos

In [2]:
# The purpose of this notebook is a final compilation of functions that will convert an output file from a MOSAIC
# dataset to graphs that can be used in publications.  There were a few key issues that needed to be resolved in
# Ben_MOSAIC_Compare, the first of which being, I no longer need to compare those datasets, as I am sticking with
# MOSAIC.  The basic additions I will need to make are:
# 1. Include an easy unit converter so that the user can input those units and have correct graphs. (done)
# 2. Include a function to fill in skipped frames in datasets without doing it manually in Excel (already done with the
#    function fillin)
# 3. Fix code to work with 1-particle datasets or exclude altogether.
# 4. Fix the error in graphs that plot one less trajectory than was calculated.

In [3]:
trajectory = np.genfromtxt('../sample_data/Slice2_2DVideo_4_22.csv',
            delimiter =",")

#Remove titles from columns
trajectory=np.delete(trajectory, 0,0)
#Remove the number column from dataset
trajectory=np.delete(trajectory,0,1)

In [4]:
def fillin(data):
    """
    This function is perfect.  It shifts the frames by the startframe and fills in any blank frames.
    """
    def startshift(data1):
        startframe = data1[0, 1]
        data1[:, 1] = data1[:, 1] - startframe
        return data1
    
    data = startshift(data)

    shap = int(max(data[:,1])) +1 
    filledin = np.zeros((shap,5))
    filledin[0,:] = data[0,:]

    count = 0
    new = 0 
    other = 0
    tot = 0

    for num in range(1, shap):
        if data[num-new,1]-data[num-new-1,1]==1:
            count = count + 1
            filledin[num, :] = data[num-new, :]
        elif data[num - new,1]-data[num - new -1,1]>1:
            new = new + 1
            iba = int(data[num - new+1,1]-data[num - new,1])
            togoin = data[num - new]
            togoin[1] = togoin[1] + 1
            filledin[num, :] = togoin
            #dataset[2] = np.insert(dataset[2], [num + new - 2], togoin, axis=0)
        
        else:
            other = other + 1
        tot = count + new + other
    
    return filledin

In [5]:
def prettify(traj, cut, lim, umppx, fps):
    """
    This function takes a trajectory dataset that has been extracted from a csv file from the MOSAIC code and augments
    it by calculating MSDs and Deffs and putting those in new columns.  The final output looks like this:
    
    Output:
    0 particle number
    1 frames
    2 x coordinate
    3 y coordinate
    4 z coordinate
    5 centered x coordinate
    6 centered y coordinate
    7 centered z coordinate
    8 3D MSD 
    9 2D xy MSD
    10 2D xz MSD
    11 2D yz MSD
    12 1D x MSD
    13 1D y MSD
    14 1D z MSD
    15 time
    16 3D Deff
    17 2D xy Deff
    18 2D xz Deff
    19 2D yz Deff
    20 1D x Deff
    21 1D y Deff
    22 1D z Deff
    
    New functionality to this code includes user inputs to define um/px defined by the microscope settings to
    convert from pixels to ums.  
    
    traj: a dataset from the MOSAIC code with the top row and first column removed.
    cut: the minimum number of frames required to be included in final dataset
    lim: the specified number of frames to be included in final dataset (often the same as cut)
    fps: frames per second
    umppx: microns per pixel
    """
    
    dataset = dict()

    particles = traj[:, 0]
    total = int(max(particles))
    total1 = total + 1
    rawdataset = traj[:, :]
    rawdataset[:, 2:5] = umppx * rawdataset[:, 2:5]
    
    # Creates an array for each trajectory containing all xyz data
    for num in range(1, total1):

        hold = np.where(particles == num)
        itindex = hold[0]
        min1 = min(itindex)
        max1 = max(itindex)
        dataset[num] = (rawdataset[min1:max1, 0:5])
    
    flee = dict()
    for num in range(1, total1):
        flee[num] = fillin(dataset[num])
    
        xmax = max(flee[num][:, 2])
        xmin = min(flee[num][:, 2])
        ymax = max(flee[num][:, 3])
        ymin = min(flee[num][:, 3])
        zmax = max(flee[num][:, 4])
        zmin = min(flee[num][:, 4])


        xc = np.array([flee[num][:, 2] - ((xmax+xmin)/2)])
        yc = np.array([flee[num][:, 3] - ((ymax+ymin)/2)])
        zc = np.array([flee[num][:, 4] - ((zmax+zmin)/2)])

        xstart = xc[0, 0]
        ystart = yc[0, 0]
        zstart = zc[0, 0]
    
        flee[num] = np.append(flee[num], xc.T, axis=1)
        flee[num] = np.append(flee[num], yc.T, axis=1)
        flee[num] = np.append(flee[num], zc.T, axis=1)
    
        the = flee[num].shape[0]
        MSD3 = np.zeros((the,1))
        M2xy = np.zeros((the,1))
        M2xz = np.zeros((the,1))
        M2yz = np.zeros((the,1))
        M1x = np.zeros((the,1))
        M1y = np.zeros((the,1))
        M1z = np.zeros((the,1))
        
        #This defines the units of time.  This is more approrpriately an input to the function.  Will fix.
        time = (1/fps) * flee[num][:, 1]
        time[0] = 0.0000000001
        time1 = np.array([time]).T

        D3 = np.zeros((the,1))
        D2xy = np.zeros((the,1))
        D2xz = np.zeros((the,1))
        D2yz = np.zeros((the,1))
        D1x = np.zeros((the,1))
        D1y = np.zeros((the,1))
        D1z = np.zeros((the,1))

        for bum in range(0, the):
            MSD3[bum, 0] = (xc[0, bum] - xstart)**2 + (yc[0, bum] - ystart)**2 + (zc[0, bum] - zstart)**2
            M2xy[bum, 0] = (xc[0, bum] - xstart)**2 + (yc[0, bum] - ystart)**2
            M2xz[bum, 0] = (xc[0, bum] - xstart)**2 + (zc[0, bum] - zstart)**2
            M2yz[bum, 0] = (yc[0, bum] - ystart)**2 + (zc[0, bum] - zstart)**2
            M1x[bum, 0] = (xc[0, bum] - xstart)**2
            M1y[bum, 0] = (yc[0, bum] - ystart)**2
            M1z[bum, 0] = (zc[0, bum] - zstart)**2
    
            D3[bum, 0] = MSD3[bum, 0]/(6*time[bum])
            D2xy[bum, 0] = M2xy[bum, 0]/(4*time[bum])
            D2xz[bum, 0] = M2xz[bum, 0]/(4*time[bum])
            D2yz[bum, 0] = M2yz[bum, 0]/(4*time[bum])
            D1x[bum, 0] = M1x[bum, 0]/(2*time[bum])
            D1y[bum, 0] = M1y[bum, 0]/(2*time[bum])
            D1z[bum, 0] = M1z[bum, 0]/(2*time[bum])
    
        flee[num] = np.append(flee[num], MSD3, axis=1)
        flee[num] = np.append(flee[num], M2xy, axis=1)
        flee[num] = np.append(flee[num], M2xz, axis=1)
        flee[num] = np.append(flee[num], M2yz, axis=1)
        flee[num] = np.append(flee[num], M1x, axis=1)
        flee[num] = np.append(flee[num], M1y, axis=1)
        flee[num] = np.append(flee[num], M1z, axis=1)
        flee[num] = np.append(flee[num], time1, axis=1)
        flee[num] = np.append(flee[num], D3, axis=1)
        flee[num] = np.append(flee[num], D2xy, axis=1)
        flee[num] = np.append(flee[num], D2xz, axis=1)
        flee[num] = np.append(flee[num], D2yz, axis=1)
        flee[num] = np.append(flee[num], D1x, axis=1)
        flee[num] = np.append(flee[num], D1y, axis=1)
        flee[num] = np.append(flee[num], D1z, axis=1)
    
    teancum = dict()
    fifties = 0
    nones = 0
    cutoff = cut

    for num in range(1, total1):
        if flee[num].shape[0] < cutoff:
            nones = nones + 1
        else:
            fifties = fifties + 1
            teancum[num - nones] = flee[num]
            # I must also redefine the particle numbers to reflect the new set.
            teancum[num - nones][:, 0] = fifties
    
    moroni = dict()
    limit = lim

    for num in range(1, fifties):
        moroni[num] = teancum[num][0:limit, :]
    
    fifties = fifties - 1
    
    return (moroni, fifties)

In [6]:
(final, tots) = prettify(trajectory, 100, 100, 0.18, 4)
tots

19

In [7]:
parts = tots
lehi = final[1]
counter = 1

while counter < parts:
    counter = counter + 1
    lehi = np.append(lehi, final[counter], axis=0)

In [8]:
# Now for a plotting function, as modified from Testing_Tracking_Software

# Creates an array 'particles' that contains the particle number at each frame.
particles = lehi[:, 0]
position = lehi[:, 5:5+4]
total = int(max(particles))
total1 = total + 1
path = dict()

# Creates an array for each trajectory containing all xyz data
for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    path[num] = (position[min1:max1, :])

# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
# ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# Plots individual trajectories
for num in range(1, total1):

    ax.plot(path[num][:, 0], path[num][:, 1], label='Particle {}'.format(num))

axbox = ax.get_position()

legunit = max(lehi[:,0])
ax.legend(loc=(0.85, 0.96-0.0273*legunit), prop={'size': 20})
ax.locator_params(nbins=6)

# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(20)

ax.title.set_fontsize(35)
ax.tick_params(direction='out', pad=3)
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f um'.format(1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f um'.format(1)))
plt.gca().set_xlim([-0.2, 0.2])
plt.gca().set_ylim([-0.2, 0.2])

# Save your figure
plt.savefig('{}.png'.format('Prelim_MOSAICTraj'), bbox_inches='tight')

plt.show()

In [9]:
# And a final function to comapare their MSDs
# Make sure to adjust tr to the same value put into prettify.

# Creates an array 'particles' that contains the particle number at each frame.
particles = lehi[:, 0]
total = int(max(particles))
total1 = total + 1
rawtime = lehi[:, 15]
rawMSD = lehi[:, 9] #17 or 9
rawMSDx = lehi[:, 12] #12 or 20
rawMSDy = lehi[:, 13] #13 or 21
MSD = dict()
MSDx = dict()
MSDy = dict()
time = dict()

tr = 100

# Creates an array for each trajectory containing all xyz data
for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    MSD[num] = (rawMSD[min1+1:min1+tr])
    MSDx[num] = (rawMSDx[min1+1:min1+tr])
    MSDy[num] = (rawMSDy[min1+1:min1+tr])
    time[num] = (rawtime[min1+1:min1+tr])

MMSD = MSD[1]
MMSDx = MSDx[1]
MMSDy = MSDy[1]

for num in range(2, total1):
    MMSD = MMSD + MSD[num]
    MMSDx = MMSDx + MSDx[num]
    MMSDy = MMSDy + MSDy[num]
    
MMSD = MMSD/total1
MMSDx = MMSDx/total1
MMSDy = MMSDy/total1

# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
# ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time[1][:], MMSD[:], label='2D MSD', linewidth=8)
ax.plot(time[1][:], MMSDx[:], label='1D MSD x', linewidth=8)
ax.plot(time[1][:], MMSDy[:], label='1D MSD y', linewidth=8)

ax.legend(loc=(0.05, 0.80), prop={'size': 40})

# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
              ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(30)

ax.title.set_fontsize(55)
ax.set_xlabel('Time (s)', fontsize=40)
ax.set_ylabel('MMSD (um2)', fontsize=40)
ax.tick_params(direction='out', pad=5)
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(4)))

# Save your figure
plt.savefig('{}.png'.format('Prelim_CompareDeffBenMOSAIC'), bbox_inches='tight')

plt.show()

In [ ]:
# The same for their Deffs
# Make sure to adjust tr to the same value put into prettify.

# Creates an array 'particles' that contains the particle number at each frame.
particles = lehi[:, 0]
total = int(max(particles))
total1 = total + 1
rawtime = lehi[:, 15]
rawMSD = lehi[:, 17] #17 or 9
rawMSDx = lehi[:, 20] #12 or 20
rawMSDy = lehi[:, 21] #13 or 21
MSD = dict()
MSDx = dict()
MSDy = dict()
time = dict()

tr = 100

# Creates an array for each trajectory containing all xyz data
for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    MSD[num] = (rawMSD[min1+1:min1+tr])
    MSDx[num] = (rawMSDx[min1+1:min1+tr])
    MSDy[num] = (rawMSDy[min1+1:min1+tr])
    time[num] = (rawtime[min1+1:min1+tr])

MMSD = MSD[1]
MMSDx = MSDx[1]
MMSDy = MSDy[1]

for num in range(2, total1):
    MMSD = MMSD + MSD[num]
    MMSDx = MMSDx + MSDx[num]
    MMSDy = MMSDy + MSDy[num]
    
MMSD = MMSD/total1
MMSDx = MMSDx/total1
MMSDy = MMSDy/total1

# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
# ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time[1][:], MMSD[:], label='2D Deff', linewidth=8)
ax.plot(time[1][:], MMSDx[:], label='1D Deffx', linewidth=8)
ax.plot(time[1][:], MMSDy[:], label='1D Deffy', linewidth=8)

ax.legend(loc=(0.70, 0.80), prop={'size': 40})

# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
              ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(30)

ax.title.set_fontsize(75)
ax.set_xlabel('Time (s)', fontsize=40)
ax.set_ylabel('Deff (um2/s)', fontsize=40)
ax.tick_params(direction='out', pad=5)
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(4)))

# Save your figure
plt.savefig('{}.png'.format('Prelim_CompareDeffBenMOSAIC'), bbox_inches='tight')

plt.show()

In [ ]:
# Now I must attempt to create code to merge datasets.  This way, I'll be able to combine data from multiple videos.
# The ultimate question I need to decide though is, how far downstream should I perform this function?  The problem
# comes in from converting pixels to microns and frames to time.  These conversions could potentially vary between
# datasets.  I could stipulate that my code MUST only be used on datasets that have the same acquisition parameters.
# This means the user must be very careful when collecting data, or it could result in inaccuracies.

#Another potential change would be to include a column of umppx and fps values.  This would require a bit of an over
#haul through many lines of code that I don't want to perform right now.  Or ever really.  Let's stick with this method
#for now.

#UPDATE: Change in plans.  I looked online, and it is considered bad practice to merge datasets.  Maybe I can perform
#this function downstream during plotting? The merging is only important when calculating averages after all.